## 3.7 日付データの処理　演習問題解答例

### 共通処理

In [1]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [2]:
# ライブラリのimport

# NumPy用ライブラリ
import numpy as np

# Matplotlib中のpyplotライブラリのインポート
import matplotlib.pyplot as plt

# matplotlib日本語化対応ライブラリのインポート
import japanize_matplotlib

# pandas用ライブラリ
import pandas as pd

# データフレーム表示用関数
from IPython.display import display

# seaborn
import seaborn as sns

In [3]:
# 表示オプション調整

# NumPy表示形式の設定
np.set_printoptions(
    suppress=True, precision=4, floatmode='fixed'
)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)

# 方眼表示ON
plt.rcParams['axes.grid'] = True


# データフレームでの表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

### CSVファイル読み込み

In [4]:
# 支払情報
url1 = 'https://github.com/makaishi2/samples/raw/main/data/rental5-jp.csv'

# 支払情報
df1 = pd.read_csv(url1, parse_dates=[4])

# 貸出日時順にソート
df2 = df1.sort_values('貸出日時')
df2 = df2.reset_index(drop=True)

### 日付集計用項目追加

In [5]:
# 集計単位計算用関数
from datetime import datetime

# 週単位の日付作成
def conv_week_day(ts):

    # 日付を年、何週目か、週の何日目かに分解
    year, week, day = ts.isocalendar()

    # その週の1日目を表現する文字列
    str = f'{year} {week} 1'

    # 文字列をdatetime型に変換
    return datetime.strptime(str, "%Y %W %w")

# 日単位の日付作成
def conv_date(ts):

    # Timestampから文字列生成
    # str: YYYY-MM-DDThh:mm:ss
    str = ts.isoformat()
 
     # 先頭10文字をdatetime型に変換
    return datetime.strptime(str[:10], '%Y-%m-%d')

In [6]:
df3 = df2.copy()

# 集計単位用の項目計算
df3.insert(4, '貸出日', df2['貸出日時'].map(conv_date))
df3.insert(5, '貸出週', df2['貸出日時'].map(conv_week_day))

# 結果確認
display(df3.head(2))

,貸出ID,DVD_ID,顧客ID,映画ID,貸出日,貸出週,貸出日時,タイトル,概要,公開年,レンタル代,カテゴリ名
0,1,367,130,80,2005-05-24,2005-05-23,2005-05-24 22:53:30,BLANKET BEVERLY,A Emotional Documentary of a Student And a Gir...,2006,2.9900,Family
1,2,1525,459,333,2005-05-24,2005-05-23,2005-05-24 22:54:33,FREAKY POCUS,A Fast-Paced Documentary of a Pastry Chef And ...,2006,2.9900,Music


### 演習問題
レンタルビデオ店では顧客離反防止施策の一環として、最近レンタルをしていない顧客をリストアップし、期間限定半額セールのDMを打つこととしました。  
顧客絞り込みの基準は「最終貸出日が2005-08-19以前」ということがすでに定まっているものとします。  
このとき、貸出情報から該当顧客を洗い出し、顧客情報を用いてDMの宛先をリスト変数形式で作成してください。  
リスト作成時は、実習で利用した貸出情報(df3)と下記の顧客情報(df5)を利用して下さい。


In [7]:
# 顧客情報
url2 = 'https://github.com/makaishi2/samples/raw/main/data/customer-jp.csv'
# 顧客情報
df5 = pd.read_csv(url2)
display(df5.head(1))

,顧客ID,店舗ID,名,姓,メール,住所ID,有効,作成日,更新日
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36.000,2021-03-06 15:53:36


In [8]:
# df3から「顧客ID」と「貸出日」の列のみを抽出
# 3.6.6項　コード3-6-13
df6 = df3[['顧客ID', '貸出日']]

# 結果確認
display(df6.head(2))

,顧客ID,貸出日
0,130,2005-05-24
1,459,2005-05-24


In [9]:
# 顧客IDごとの最終貸出日を求める
# (ヒント)顧客ごとグループ化して「貸出日」の最大値を求めればいい
# 3.7.3項　コード3-7-5
df7 = df6.groupby('顧客ID').max()

# 顧客IDをデータフレームの列に戻す
# 3.6節演習問題　コード3-6-20
df8 = df7.reset_index()

# 結果確認
display(df8.head(2))

,顧客ID,貸出日
0,1,2005-08-22
1,2,2005-08-23


In [10]:
# 最終貸出日が 2005-08-19以前の顧客を絞り込む

# 基準日の定義
# 3.7.5項　コード3-7-10
ldate = pd.to_datetime('2005-08-19')

# queryメソッドで検索
# 3.7.5項　コード3-7-10
df9 = df8.query('貸出日 <= @ldate')

# 結果確認
display(df9)

,顧客ID,貸出日
98,99,2005-08-19
207,208,2005-08-17
325,326,2005-08-19
329,330,2005-08-19
427,428,2005-08-17
482,483,2005-08-19
484,485,2005-08-19
497,498,2005-08-19
572,573,2005-08-19


In [11]:
# メールアドレス一覧の作成

# df5から顧客ID、メールアドレスを抽出
# 3.6.7項　コード3-6-15
df10 = df5[['顧客ID', 'メール']]

# df9と結合
# 3.6.7項　コード3-6-17
df11 = pd.merge(df9, df10, on='顧客ID')

# 結果確認
display(df11.head(2))

,顧客ID,貸出日,メール
0,99,2005-08-19,EMILY.DIAZ@sakilacustomer.org
1,208,2005-08-17,LUCY.WHEELER@sakilacustomer.org


In [12]:
# メールアドレスをリスト形式で抽出
# 2.4.3項　コード2-4-5
mlist = list(df11['メール'])

# 結果確認
print(mlist)

['EMILY.DIAZ@sakilacustomer.org', 'LUCY.WHEELER@sakilacustomer.org', 'JOSE.ANDREW@sakilacustomer.org', 'SCOTT.SHELLEY@sakilacustomer.org', 'HERBERT.KRUGER@sakilacustomer.org', 'VERNON.CHAPA@sakilacustomer.org', 'CLYDE.TOBIAS@sakilacustomer.org', 'GENE.SANBORN@sakilacustomer.org', 'BYRON.BOX@sakilacustomer.org']
